# Задача на анализ продукта 
Есть некий продукт «Technology». Существует несколько вариантов подключения продукта пользователем:
- Отдельно, только «Technology»
- В связке с вторым продуктом «Service»: оплачивать единый тариф, включающий в себя оба продукта (будем такое подключение называть – Combo).

Второй вариант подключения считается приоритетным, поскольку в перспективе клиент использует больше услуг, дольше остается пользователем, имеет меньшую вероятность отключиться. Поэтому Combo подключение продается со скидкой таким образом, что это подключение становится даже выгоднее, чем только «Technology». У сотрудников продаж есть цель на подключение именно Combo.
Есть гипотеза, что сотрудники продают Combo клиентам, которым продукт «Service» не нужен. Тем самым несут компании больше расходов на скидках, а также насыщают клиентскую базу продукта «Service» неактивными людьми, фейковыми клиентами.

**Описание данных:**
    

- ***technology*** - Объект с клиентами активными* в продукте «Technology» 
  - date_key_src - Период	
  - client_id_tech - ID Клиента в продукте Technology	
  - combo_ind	- Признак Combo 
  - client_id_serv - ID Клиента в продукте Service	


- ***service*** - Объект с клиентами активными* в продукте «Service»
  - time_key_dt -	Период	
  - client_id_serv -	ID Клиента в продукте 
  - segment -	Сегмент активности клиента	
  - sales_ind -	Признак новых продаж (sales\old)	


- ***service_revenue*** -	Объект с доходами по клиентам по продукту «Service»
  - time_key_dt -	Период	
  - client_id_serv -	ID Клиента в продукте Service	
  - revenue -	Доход в условных единицах	


- ***service_fraud*** - Объект с абонентами «Service» в статусе Фрод
  - time_key_dt -	Период	
  - client_id_serv -	ID Клиента в продукте Service	
  - channel -	Канал подключения Фрода	


**Необходимо ответить на вопросы:** 
- Какую динамику показывают базы клиентов «Technology», «Service» и Combo в частности?
- Как абоненты Combo ведут себя с точки зрения продукта «Service»? Насколько стабильно пользуются и сколько за «Service» платят?
- Динамика Combo Продаж. Есть ли среди абонентов FMC продаж – Фрод? Источник фрода?
- Исследовать качество продаж Combo: 
    - Как подключенные клиенты выживают (остаются в базе клиентов) на 1- 4Месяц, 12Месяц с точки зрения продукта «Service»? С точки зрения «Technology»?


## Предобработка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np

pd.options.display.max_columns = 30
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
technology = pd.read_csv('technology_subs.csv') 
service = pd.read_csv('service_subs.csv') 
service_revenue = pd.read_csv('service_revenue.csv') 
service_fraud = pd.read_csv('service_fraud.csv')

In [3]:
# Функция первичного просмотра
def first_view(df, tab_name):
    print(f'Таблица {tab_name}')
    print('--------------------------------------------------')
    df.info()
    print('--------------------------------------------------')
    display(df.head())
    print('--------------------------------------------------')
    print('Количество дубликатов:', df.duplicated().sum())
    print('--------------------------------------------------')
    for col in df.columns:
        if len(df[col].unique())<10:
            print(f'Соотношение в столбце {col} в %')
            print('--------------------------------------------------')
            print(df[col].value_counts(normalize=True))
            print('--------------------------------------------------')
        else:
            if df[col].dtype in ['float', 'int']:
                print('Распределение в столбце', col)
                print('--------------------------------------------------')
                print(df[col].describe())
                print('--------------------------------------------------')

### Таблица technology

In [4]:
first_view(technology, 'technology')

Таблица technology
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000968 entries, 0 to 1000967
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   date_key_src    1000968 non-null  object
 1   client_id_tech  1000968 non-null  object
 2   combo_ind       1000968 non-null  int64 
 3   client_id_serv  570589 non-null   object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB
--------------------------------------------------


,date_key_src,client_id_tech,combo_ind,client_id_serv
0,2022-02-01,A694897209,1,A64955227227421
1,2021-06-01,A694898225,1,A64900848624998
2,2021-07-01,A694900227,1,A68699999951359
3,2021-10-01,A694902219,1,A64955426727513
4,2022-05-01,A694903225,0,NaN


--------------------------------------------------
Количество дубликатов: 0
--------------------------------------------------
Соотношение в столбце combo_ind в %
--------------------------------------------------
1   0.60
0   0.40
Name: combo_ind, dtype: float64
--------------------------------------------------


In [5]:
# Изменим названия столбцов и тип данных в date_key_src, чтобы было удобнее работать с таблицей
technology.columns = ['dt', 'id_tech', 'combo', 'id_serv']
technology.dt = pd.to_datetime(technology.dt, format='%Y-%m-%d')

In [6]:
# Даты
min_date = technology.dt.min()
max_date = technology.dt.max()
dates = pd.date_range(start=min_date, end=max_date)
print('Минимальная дата', min_date)
print('Максимальная дата', max_date)
print(f'Изучаем данные за период {len(dates)} дней')
technology.dt.value_counts().reset_index().sort_values(by='index')

Минимальная дата 2021-01-01 00:00:00
Максимальная дата 2022-07-01 00:00:00
Изучаем данные за период 547 дней


,index,dt
17,2021-01-01,51312
18,2021-02-01,51286
16,2021-03-01,51492
11,2021-04-01,51976
12,2021-05-01,51887
15,2021-06-01,51718
13,2021-07-01,51761
14,2021-08-01,51749
10,2021-09-01,52577
9,2021-10-01,52948


Изучаемый период 547 дней (19 месяцев). Даты обрезаны до месяца.

In [7]:
# Посмотрим на пропуски в столбце id_serv
display(technology[technology.id_serv.isna()].sample(5))

print('Соотношение в столбце combo в %, для строк где id_serv=NaN')
display(technology[technology.id_serv.isna()].combo.value_counts(normalize=True))

cnt_comboid_nan = len(technology[(technology.id_serv.isna()) & (technology.combo==1)])
print(f"Количество комбо подключений без id_serv: {cnt_comboid_nan}")  

,dt,id_tech,combo,id_serv
59279,2022-01-01,K6977273734,0,NaN
577419,2021-11-01,K6982827288,0,NaN
52632,2021-08-01,I73418650351,0,NaN
730037,2022-03-01,S70197033655,0,NaN
147843,2021-03-01,O73195143727,1,NaN


Соотношение в столбце combo в %, для строк где id_serv=NaN


0   0.94
1   0.06
Name: combo, dtype: float64

Количество комбо подключений без id_serv: 26727


Для большинства пропусков характерно "соло" подключение, для остальных по каким-то причинам не подгрузились id_serv.
Попробуем найти id_serv через дублирующиеся id_tech

In [8]:
# Дубли
print('Количество уникальных id_tech:', len(technology.id_tech.unique()))
print('Количество не уникальных id_tech:', len(technology)-len(technology.id_tech.unique()))

# ID дублей
id_tech_dub = technology[technology.id_tech.duplicated()].id_tech

# Фильтруем датафрейм
technology_dub = (technology[technology.id_tech.isin(id_tech_dub)]
                       .sort_values(by=['id_tech', 'dt'])).reset_index(drop=True)
technology_dub.head(20)

Количество уникальных id_tech: 989045
Количество не уникальных id_tech: 11923


,dt,id_tech,combo,id_serv
0,2021-03-01,A694988668,1,NaN
1,2021-11-01,A694988668,1,A66937424337215
2,2021-03-01,A694996711,1,NaN
3,2021-11-01,A694996711,1,A64668633012710
4,2021-03-01,A6950721139,0,NaN
5,2021-05-01,A6950721139,1,A68689531250603
6,2022-01-01,A6979885525,1,A6452499713760
7,2022-01-01,A6979885525,1,A6452499713761
8,2022-04-01,A6983657316,1,A68761534056692
9,2022-06-01,A6983657316,0,NaN


В базе очень мало дублирующихся id_tech, получается в рамках одного пользователя в подавляющем числе случаев лайфтайм будет = 1 месяцу. Возможно не полностью выгрузились данные. Отвечать на последний вопрос придется исходя их того количества которое имеется.  

Видим, что для некоторых пар id_tech значение комбо различается, такое возможно например при переходе от соло подключения к комбо и наоборот. Чтобы найти нужные нам id_serv необходим отфильтровать пары строк для которых комбо всегда = 1.

In [9]:
# Для каждого id находим сумму статуса combo и количество строк с одинаковым id_tech
id_combo1 = technology_dub.groupby('id_tech', as_index=False).agg({'combo':'sum', 'dt':'count'})

# Делим количество строк на сумму комбо
id_combo1['sign'] = id_combo1['dt']/id_combo1['combo']

# Убираем лишние строки, нам нужны только те, где значение признака = 1
id_combo1 = id_combo1[id_combo1.sign==1].id_tech

# Составим словарь для замены пропущенных значений
dict_replace = technology_dub[technology_dub.id_tech.isin(id_combo1)][['id_tech', 'id_serv']].dropna().drop_duplicates()

# Заменим значения в пропущенных строках на значения из словаря
technology = technology.fillna(dict_replace)
technology.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000968 entries, 0 to 1000967
Data columns (total 4 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   dt       1000968 non-null  datetime64[ns]
 1   id_tech  1000968 non-null  object        
 2   combo    1000968 non-null  int64         
 3   id_serv  574113 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 30.5+ MB


Обработали более 3500 пропущеных значений в столбце id_serv.

### Таблица service

In [10]:
first_view(service, 'service_subs')

Таблица service_subs
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1569621 entries, 0 to 1569620
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   time_key_dt     1569621 non-null  object
 1   client_id_serv  1569621 non-null  object
 2   segment         1569621 non-null  object
 3   sales_ind       1569621 non-null  object
dtypes: object(4)
memory usage: 47.9+ MB
--------------------------------------------------


,time_key_dt,client_id_serv,segment,sales_ind
0,2021-01-01,A64309659750,1,old
1,2021-01-01,A643510019114,1,old
2,2021-01-01,A643511179115,1,old
3,2021-01-01,A644422566295,1,old
4,2021-01-01,A644424051299,1,old


--------------------------------------------------
Количество дубликатов: 0
--------------------------------------------------
Соотношение в столбце segment в %
--------------------------------------------------
1     0.74
2     0.20
New   0.05
Name: segment, dtype: float64
--------------------------------------------------
Соотношение в столбце sales_ind в %
--------------------------------------------------
old     0.97
sales   0.03
Name: sales_ind, dtype: float64
--------------------------------------------------


In [11]:
# Изменим названия столбцов и типы данных, чтобы было удобнее работать с таблицей
service.columns = ['dt', 'id_serv', 'segment', 'sales']

In [12]:
service.loc[service.segment=='New', 'segment'] = '3'
service.segment = service.segment.astype(int)

In [13]:
service['dt'] = pd.to_datetime(service['dt'], format='%Y-%m-%d')

In [14]:
# Дата
min_date = service.dt.min()
max_date = service.dt.max()
dates = pd.date_range(start=min_date, end=max_date)
print('Минимальная дата', min_date)
print('Максимальная дата', max_date)
print(f'Изучаем данные за период {len(dates)} дней')
service.dt.value_counts().reset_index().sort_values(by='index')

Минимальная дата 2021-01-01 00:00:00
Максимальная дата 2022-08-01 00:00:00
Изучаем данные за период 578 дней


,index,dt
17,2021-01-01,77997
18,2021-02-01,77698
16,2021-03-01,78378
15,2021-04-01,78849
14,2021-05-01,79743
13,2021-06-01,79803
9,2021-07-01,80272
6,2021-08-01,80662
1,2021-09-01,81314
5,2021-10-01,80696


Изучаемый период на 1 месяц больше, чем для таблицы technology (20 месяцев). Даты также обрезаны. За последний месяц данные сильно отличаются, возможно была не полная выгрузка, этот месяц идет под удаление. 

In [15]:
# Дубли
print('Количество не уникальных id_tech:', len(service)-len(service.id_serv.unique()))

Количество не уникальных id_tech: 375


Тут количество парных id совсем малое, на последний вопрос будем отвечать только исходя из парных строк в технолоджи.

### Таблица service_revenue

In [16]:
first_view(service_revenue, 'service_revenue')

Таблица service_revenue
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1569579 entries, 0 to 1569578
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   time_key_dt     1569579 non-null  object 
 1   client_id_serv  1569579 non-null  object 
 2   revenue         1569579 non-null  float64
dtypes: float64(1), object(2)
memory usage: 35.9+ MB
--------------------------------------------------


,time_key_dt,client_id_serv,revenue
0,2021-04-01,V644328120231,382.72
1,2021-04-01,K69097158072550,377.43
2,2021-04-01,S65849265234609,682.29
3,2021-04-01,U68598416040936,60.96
4,2021-04-01,S68677155648563,434.01


--------------------------------------------------
Количество дубликатов: 0
--------------------------------------------------
Распределение в столбце revenue
--------------------------------------------------
count   1,569,579.00
mean          318.44
std           282.74
min        -3,992.84
25%           128.93
50%           280.69
75%           434.60
max        26,718.27
Name: revenue, dtype: float64
--------------------------------------------------


In [17]:
# Изменим названия столбцов и тип данных, чтобы было удобнее работать с таблицей
service_revenue.columns = ['dt', 'id_serv', 'revenue']
service_revenue.dt = pd.to_datetime(service_revenue.dt, format='%Y-%m-%d')

In [18]:
# Дата
min_date = service_revenue.dt.min()
max_date = service_revenue.dt.max()
dates = pd.date_range(start=min_date, end=max_date)
print('Минимальная дата', min_date)
print('Максимальная дата', max_date)
print(f'Изучаем данные за период {len(dates)} дней')
service_revenue.dt.value_counts().reset_index().sort_values(by='index')

Минимальная дата 2021-01-01 00:00:00
Максимальная дата 2022-08-01 00:00:00
Изучаем данные за период 578 дней


,index,dt
17,2021-01-01,77992
18,2021-02-01,77697
16,2021-03-01,78372
15,2021-04-01,78858
14,2021-05-01,79741
13,2021-06-01,79802
9,2021-07-01,80271
6,2021-08-01,80660
1,2021-09-01,81310
5,2021-10-01,80692


Период точно такой же как и в таблице service (20 месяцев). Данные за 2022-08-01 под удаление.

В столбце revenue присутствуют отрицательные значения. Возможно обещаный платеж или что то в этом духе.

### Таблица service_fraud

In [19]:
first_view(service_fraud, 'service_fraud')

Таблица service_fraud
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700 entries, 0 to 2699
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   time_key_dt     2700 non-null   object
 1   client_id_serv  2700 non-null   object
 2   channel         2700 non-null   object
dtypes: object(3)
memory usage: 63.4+ KB
--------------------------------------------------


,time_key_dt,client_id_serv,channel
0,2022-07-01,T644930978638,Alternative
1,2021-03-01,K68685073049356,Alternative
2,2021-03-01,G68827227560004,Alternative
3,2021-03-01,K64784666024594,Alternative
4,2021-06-01,T68631562544549,Alternative


--------------------------------------------------
Количество дубликатов: 0
--------------------------------------------------
Соотношение в столбце channel в %
--------------------------------------------------
Alternative   0.74
Office        0.15
Franchise     0.11
Name: channel, dtype: float64
--------------------------------------------------


In [20]:
# Изменим названия столбцов и типы данных, чтобы было удобнее работать с таблицей
service_fraud.columns = ['dt', 'id_serv', 'channel']
service_fraud.dt = pd.to_datetime(service_fraud.dt, format='%Y-%m-%d')

In [21]:
min_date = service_fraud.dt.min()
max_date = service_fraud.dt.max()
dates = pd.date_range(start=min_date, end=max_date)
print('Минимальная дата', min_date)
print('Максимальная дата', max_date)
print('Изучаем данные за период {} дней'.format(len(dates)))
service_revenue.dt.value_counts().reset_index().sort_values(by='index')

Минимальная дата 2021-01-01 00:00:00
Максимальная дата 2022-08-01 00:00:00
Изучаем данные за период 578 дней


,index,dt
17,2021-01-01,77992
18,2021-02-01,77697
16,2021-03-01,78372
15,2021-04-01,78858
14,2021-05-01,79741
13,2021-06-01,79802
9,2021-07-01,80271
6,2021-08-01,80660
1,2021-09-01,81310
5,2021-10-01,80692


Временой период так же совпадает с 2умя продидущими датафреймами.

### Обьединение датафреймов

In [22]:
# Обьединяем датафреймы
service_full = service.merge(technology[['dt', 'id_serv', 'combo']], on=['id_serv', 'dt'], how='left')\
                      .merge(service_revenue, on=['id_serv', 'dt'], how='left')\
                      .merge(service_fraud, on=['id_serv', 'dt'], how='left')

# Обрезаем последний месяц
service_full = service_full[service_full.dt < '2022-08-01']

In [23]:
service_full.dt.value_counts().reset_index().sort_values(by='index')

,index,dt
17,2021-01-01,78005
18,2021-02-01,77704
16,2021-03-01,78384
15,2021-04-01,78869
14,2021-05-01,79750
13,2021-06-01,79807
9,2021-07-01,80280
6,2021-08-01,80666
1,2021-09-01,81335
5,2021-10-01,80706


In [24]:
service_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520923 entries, 0 to 1568075
Data columns (total 7 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   dt       1520923 non-null  datetime64[ns]
 1   id_serv  1520923 non-null  object        
 2   segment  1520923 non-null  int32         
 3   sales    1520923 non-null  object        
 4   combo    570771 non-null   float64       
 5   revenue  1520863 non-null  float64       
 6   channel  2597 non-null     object        
dtypes: datetime64[ns](1), float64(2), int32(1), object(3)
memory usage: 87.0+ MB


In [25]:
# Заменим пропущенные значения в столбце combo на 0 
service_full.loc[service_full.combo.isna(), 'combo'] = '0' 
service_full.combo = service_full.combo.astype(int)

In [26]:
service_full.head()

,dt,id_serv,segment,sales,combo,revenue,channel
0,2021-01-01,A64309659750,1,old,0,"1,889.86",NaN
1,2021-01-01,A643510019114,1,old,1,365.72,NaN
2,2021-01-01,A643511179115,1,old,1,390.79,NaN
3,2021-01-01,A644422566295,1,old,0,85.38,NaN
4,2021-01-01,A644424051299,1,old,1,268.94,NaN


##  Ответы на вопросы
### Какую динамику показывают базы клиентов «Technology», «Service» и Combo в частности?

In [27]:
# База клиентов «Technology»
pt_tech = technology[technology.combo==0].groupby('dt', as_index=False)['id_tech'].count()\
                    .rename(columns={'id_tech':'cnt_tech'})
# База клиентов «Combo»
pt_combo = technology[technology.combo==1].groupby('dt', as_index=False)['id_tech'].count()\
                    .rename(columns={'id_tech':'cnt_combo'})
# База клиентов «Service» (не забываем обрезать лишний месяц)
pt_serv = service_full[(service_full.combo==0) & (service_full.dt<='2022-07-01')]\
                    .groupby('dt', as_index=False)['id_serv'].count()\
                    .rename(columns={'id_serv':'cnt_serv'})

# Итоговая сводная таблица
pt_qu1 = pt_serv.merge(pt_combo, on='dt', how='left').merge(pt_tech, on='dt', how='left') 
pt_qu1

,dt,cnt_serv,cnt_combo,cnt_tech
0,2021-01-01,50154,29062,22250
1,2021-02-01,49673,29152,22134
2,2021-03-01,50279,29281,22211
3,2021-04-01,50115,29943,22033
4,2021-05-01,50773,30182,21705
5,2021-06-01,50607,30466,21252
6,2021-07-01,50817,30783,20978
7,2021-08-01,51222,30784,20965
8,2021-09-01,51247,31415,21162
9,2021-10-01,50385,31695,21253


### Как абоненты Combo ведут себя с точки зрения продукта «Service»? Насколько стабильно пользуются и сколько за «Service» платят? 

In [28]:
# Производим группировку по необходимым признакам
pt_qu2 = service_full.pivot_table(index=['dt', 'segment', 'sales', 'combo'], 
                                  aggfunc={'id_serv':'count', 'revenue':['mean', 'sum']}).reset_index()

pt_qu2.columns = ['dt', 'segment', 'sales', 'combo', 'cnt_id', 'mean_rev', 'sum_rev']

In [29]:
pt_qu2

,dt,segment,sales,combo,cnt_id,mean_rev,sum_rev
0,2021-01-01,1,old,0,31872,329.36,"10,496,992.79"
1,2021-01-01,1,old,1,24239,409.48,"9,925,421.58"
2,2021-01-01,2,old,0,14361,117.55,"1,687,733.34"
3,2021-01-01,2,old,1,2863,288.90,"827,120.93"
4,2021-01-01,3,old,0,1974,168.84,"333,281.34"
...,...,...,...,...,...,...,...
164,2022-07-01,2,sales,0,18,111.45,"2,006.13"
165,2022-07-01,3,old,0,1301,230.57,"299,973.52"
166,2022-07-01,3,old,1,458,275.63,"126,236.91"
167,2022-07-01,3,sales,0,1267,181.91,"230,474.16"


### Динамика Combo Продаж. Есть ли среди абонентов FMC продаж – Фрод? Источник фрода?

In [30]:
# Обрезаем лишние строки
frod = service_full[service_full.channel.notna()]

# Произзводим группировку по необходимым признакам
pt_qu3 = frod.pivot_table(index=['dt', 'channel', 'combo'], 
                          aggfunc={'id_serv':'count', 'revenue': 'sum'})\
                          .reset_index().rename(columns={'id_serv':'cnt_id', 'revenue': 'sum_rev'})

In [31]:
pt_qu3

,dt,channel,combo,cnt_id,sum_rev
0,2021-01-01,Alternative,0,74,"12,599.41"
1,2021-01-01,Alternative,1,5,822.99
2,2021-01-01,Franchise,0,26,"4,505.36"
3,2021-01-01,Franchise,1,3,470.92
4,2021-01-01,Office,0,21,"8,523.70"
...,...,...,...,...,...
100,2022-07-01,Alternative,0,56,"8,106.93"
101,2022-07-01,Alternative,1,8,"1,954.44"
102,2022-07-01,Franchise,0,10,"1,602.42"
103,2022-07-01,Office,0,9,"2,192.67"


### Исследовать качество продаж Combo: 
  - Как подключенные клиенты выживают (остаются в базе клиентов) на 1- 4Месяц, 12Месяц с точки зрения продукта «Service»? С точки зрения «Technology»?

In [32]:
# Вычисляем минимальную и максимальную дату.
lifetime = technology_dub.groupby('id_tech', as_index=False).agg({'dt':['min','max']})
lifetime.columns=['id_tech','min_dt','max_dt']

# Вычисляем разницу в месяцах
lifetime['lifetime_mon'] = lifetime['max_dt'] - lifetime['min_dt']
lifetime['lifetime_mon'] = lifetime['lifetime_mon'].astype("timedelta64[M]").astype(int)

# Оставляем нужные столбцы
lifetime = lifetime[['id_tech', 'lifetime_mon']]

In [33]:
# Добавляем значения лайфтайма в общий датафрейм
technology_lt = technology.merge(lifetime, on='id_tech')

# Вычисляем количество клиентов для каждого лайфтайма
pt_qu4 = technology_lt.pivot_table(index='lifetime_mon', columns='combo', values='id_tech', aggfunc='count')\
                      .sort_values(by='lifetime_mon', ascending=False).reset_index()
pt_qu4.columns = ['lifetime_mon', 'Service', 'Combo']

# Общее количество клиентов для каждого лайфтайма
pt_qu4['Service_cum'] = pt_qu4['Service'].cumsum()
pt_qu4['Combo_cum'] = pt_qu4['Combo'].cumsum()
pt_qu4

,lifetime_mon,Service,Combo,Service_cum,Combo_cum
0,17,135,240,135,240
1,16,194,252,329,492
2,15,101,149,430,641
3,14,94,135,524,776
4,13,205,331,729,1107
5,12,19,28,748,1135
6,11,111,187,859,1322
7,10,84,135,943,1457
8,9,135,285,1078,1742
9,8,162,332,1240,2074


## SQL Скрипты

В написании скриптов использовал уже измененные названия колонок. Для удобства буду выводить первую строку используемых в скрипте таблиц.

### Какую динамику показывают базы клиентов «Technology», «Service» и Combo в частности?

``` SQL
-- делаем отдельные сводные таблицы для каждого типа подключения
WITH t1 AS (
SELECT date_trunc('month', "dt")::date AS dt_month,
       count("id_tech") AS cnt_tech
FROM technology
WHERE combo != 1
GROUP BY dt_month),

t2 AS (
SELECT date_trunc('month', "dt")::date AS dt_month,
       count("id_tech") AS cnt_combo
FROM technology
WHERE combo = 1
GROUP BY dt_month),

t3 AS (
SELECT date_trunc('month', s.dt)::date AS dt_month,
       count(s.id_serv) AS cnt_serv
FROM service s 
LEFT JOIN technology t ON (s.id_serv=t.id_serv AND s.dt=t.dt)
WHERE s.dt < '2022-08-01' AND combo != 1
GROUP BY dt_month)

--обьединяем в общую сводную таблицу
SELECT t1.dt_month, cnt_tech, cnt_combo, cnt_serv
FROM t1
JOIN t2 ON t1.dt_month=t2.dt_month
JOIN t3 ON t1.dt_month=t3.dt_month;

```

In [34]:
# Используемые таблицы
display(technology.head(1))
display(service.head(1))

,dt,id_tech,combo,id_serv
0,2022-02-01,A694897209,1,A64955227227421


,dt,id_serv,segment,sales
0,2021-01-01,A64309659750,1,old


In [35]:
# Итоговая сводная таблица по результатам запроса
pt_qu1

,dt,cnt_serv,cnt_combo,cnt_tech
0,2021-01-01,50154,29062,22250
1,2021-02-01,49673,29152,22134
2,2021-03-01,50279,29281,22211
3,2021-04-01,50115,29943,22033
4,2021-05-01,50773,30182,21705
5,2021-06-01,50607,30466,21252
6,2021-07-01,50817,30783,20978
7,2021-08-01,51222,30784,20965
8,2021-09-01,51247,31415,21162
9,2021-10-01,50385,31695,21253


### Как абоненты Combo ведут себя с точки зрения продукта «Service»? Насколько стабильно пользуются и сколько за «Service» платят? 

```SQL

SELECT date_trunc('month', s.dt)::date AS dt_month,
       s.segment,
       s.sales,
       t.combo,
       count(s.id_serv) AS cnt_serv,
       avg(r.revenue) AS mean_rev,
       sum(r.revenue) AS sum_rev
FROM service s 
-- использую левое обьединение, так как возможно не для всех айди есть признак комбо или значение выручки
LEFT JOIN technology t ON (s.id_serv=t.id_serv AND s.dt=t.dt) 
LEFT JOIN service_revenue r ON (s.id_serv=r.id_serv AND s.dt=r.dt)
GROUP BY dt_month, segment, sales, combo
```

In [36]:
# Используемые таблицы
display(technology.head(1))
display(service.head(1))
display(service_revenue.head(1))

,dt,id_tech,combo,id_serv
0,2022-02-01,A694897209,1,A64955227227421


,dt,id_serv,segment,sales
0,2021-01-01,A64309659750,1,old


,dt,id_serv,revenue
0,2021-04-01,V644328120231,382.72


In [37]:
# Итоговая сводная таблица по результатам запроса
pt_qu2

,dt,segment,sales,combo,cnt_id,mean_rev,sum_rev
0,2021-01-01,1,old,0,31872,329.36,"10,496,992.79"
1,2021-01-01,1,old,1,24239,409.48,"9,925,421.58"
2,2021-01-01,2,old,0,14361,117.55,"1,687,733.34"
3,2021-01-01,2,old,1,2863,288.90,"827,120.93"
4,2021-01-01,3,old,0,1974,168.84,"333,281.34"
...,...,...,...,...,...,...,...
164,2022-07-01,2,sales,0,18,111.45,"2,006.13"
165,2022-07-01,3,old,0,1301,230.57,"299,973.52"
166,2022-07-01,3,old,1,458,275.63,"126,236.91"
167,2022-07-01,3,sales,0,1267,181.91,"230,474.16"


### Динамика Combo Продаж. Есть ли среди абонентов FMC продаж – Фрод? Источник фрода?

Динамику Combo продаж можно вычислить с помощью предидущей таблицы.

```SQL

SELECT date_trunc('month', s.dt)::date AS dt_month,
       f.channel,
       t.combo,
       count(r.id_serv) AS cnt_serv,
       sum(r.revenue) AS sum_rev
FROM service_revenue r
-- использую внутреннее обьединение, чтобы убрать строки где абоненты не Фрод
JOIN service_fraud f ON (r.id_serv=f.id_serv AND r.dt=f.dt)
-- использую левое обьединение, так как возможно не для всех айди есть признак комбо
LEFT JOIN technology t ON (r.id_serv=t.id_serv AND r.dt=t.dt) 
GROUP BY dt_month, f.channel, t.combo
```

In [38]:
# Используемые таблицы
display(technology.head(1))
display(service_fraud.head(1))
display(service_revenue.head(1))

,dt,id_tech,combo,id_serv
0,2022-02-01,A694897209,1,A64955227227421


,dt,id_serv,channel
0,2022-07-01,T644930978638,Alternative


,dt,id_serv,revenue
0,2021-04-01,V644328120231,382.72


In [39]:
# Итоговая сводная таблица по результатам запроса
pt_qu3

,dt,channel,combo,cnt_id,sum_rev
0,2021-01-01,Alternative,0,74,"12,599.41"
1,2021-01-01,Alternative,1,5,822.99
2,2021-01-01,Franchise,0,26,"4,505.36"
3,2021-01-01,Franchise,1,3,470.92
4,2021-01-01,Office,0,21,"8,523.70"
...,...,...,...,...,...
100,2022-07-01,Alternative,0,56,"8,106.93"
101,2022-07-01,Alternative,1,8,"1,954.44"
102,2022-07-01,Franchise,0,10,"1,602.42"
103,2022-07-01,Office,0,9,"2,192.67"


### Исследовать качество продаж Combo: 
  - Как подключенные клиенты выживают (остаются в базе клиентов) на 1- 4Месяц, 12Месяц с точки зрения продукта «Service»? С точки зрения «Technology»?


``` SQL
-- Вычисляем лайфтайм
WITH combo_lt AS
  
  (SELECT id_tech, 
   -- Отнимаю от большей даты меньшую, вынимаю количество дней, делю на 30, а затем округляю в меньшую сторону
          floor(extract(DAY FROM (max(dt) - min(dt))) / 30) AS lt_mon
   FROM technology
   WHERE combo = 1),
-- Группируем пользователей по лайфтайму  
   pt_combo_lt AS
  (SELECT lt_mon,
          count(id_tech) AS cnt_combo_id
   FROM combo_lt
   GROUP BY lt_mon),
-- Добавляем сумму пользователей с накоплением     
   pt_combo_lt_cum AS
  (SELECT *, sum(cnt_combo_id) over(ORDER BY lt_mon DESC) AS cum_combo_id
   FROM pt_combo_lt), 
   
--Проделываем те же операции для пользователей technology   
   tech_lt AS
  (SELECT id_tech,
          floor(extract(DAY FROM (max(dt) - min(dt))) / 30) AS lt_mon
   FROM technology
   WHERE combo = 0),
   
   pt_tech_lt AS
  (SELECT lt_mon,
          count(id_tech) AS cnt_tech_id
   FROM tech_lt
   GROUP BY lt_mon),
   
   pt_tech_lt_cum AS
  (SELECT *,
          sum(cnt_tech_id) over(ORDER BY lt_mon DESC)
   FROM pt_tech_lt),

--Обьединяем в общую сводную таблицу
SELECT c.lt_mon,
       cnt_combo_id,
       cum_combo_id,
       cnt_tech_id,
       cum_tech_id
FROM pt_combo_lt_cum c
JOIN pt_tech_lt_cum t ON c.lt_mon=t.lt_mon 
WHERE c.lt_mon <= 12;
```

In [40]:
# Используемые таблицы
display(technology.head(1))
display(service.head(1))

,dt,id_tech,combo,id_serv
0,2022-02-01,A694897209,1,A64955227227421


,dt,id_serv,segment,sales
0,2021-01-01,A64309659750,1,old


In [41]:
# Итоговая сводная таблица по результатам запроса
pt_qu4

,lifetime_mon,Service,Combo,Service_cum,Combo_cum
0,17,135,240,135,240
1,16,194,252,329,492
2,15,101,149,430,641
3,14,94,135,524,776
4,13,205,331,729,1107
5,12,19,28,748,1135
6,11,111,187,859,1322
7,10,84,135,943,1457
8,9,135,285,1078,1742
9,8,162,332,1240,2074


## Выгружаю данные

pt_qu1.to_csv('pt_qu1.csv')
pt_qu2.to_csv('pt_qu2.csv')
pt_qu3.to_csv('pt_qu3.csv')
pt_qu4.to_csv('pt_qu4.csv')